# Truth Inference from Crowdsourcing with Optimal Return of Interest: A Strategic Macro Assignment and Micro Optimization Paradigm

## Table of content
- [Setup](#setup)
    - [Import](#import)
    - [Parameters](#parameters)
- [MAMO](#MAMO)
    - [Mirco](#micro)
        - [Utility Functions](#micro-utility)
        - [micro cp](#micro-cp)
        - [micro greedy](#micro-greedy)
        - [micro mckp](#micro-mckp)
        - [micro difficulty](#micro-difficulty)
    - [Macro](#macro)
        - [Utility Functions](#macro-utility)
        - [macro without multi-process](#macro-without-multi-process)
        - [macro with multi-process](#macro-with-multi-process)
- [Random-MAMO](#random-MAMO)
    - [Micro](#micro-random)
    - [Macro](#macro-random)
- [Requallo](#requallo)
- [Experiment](#experiment)

---

<a name='setup'></a>
# Setup

<a name='import'></a>
## Import

In [ ]:
import os
import sys
import copy
import time
import random
import logging
from faker import Faker
from collections import Counter
from copy import deepcopy
from math import ceil
from itertools import combinations
from pprint import pprint
from multiprocessing import Pool

import ujson
import numpy as np

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
fake_gen = Faker()
fake_gen.seed(42)

random.seed(42)

<a name='parameters'></a>
## Parameters

In [ ]:
# For micro
ALPHA = 0.8
MAX_ITERATION = 100
THRESHOLD = 0.75

## For difficulty
ETA = 0.3

# For macro
SIMULATE_CNT = 5
MAX_ROUND = 5
CONF_THRESHOLD = 0.3

# For overall setup
max_assign_cnt = 3
budget = 500

-----

<a name='MAMO'></a>
# MAMO

<a name='micro'></a>
## Micro

<a name='micro-utility'></a>
### Utility Functions

In [ ]:
def gen_difficulty(task):
    """
    利用機率的方法決定 difficulty，把 pre-confidence 的左邊跟右邊個切三段，當作標準差，
    並在左右邊各用不同的標準差隨機 Random 出 200 個數值，在 pre-confidence 的左右邊各取 50 個數值，
    最後在 100 個數值內隨機挑選出一個當作 difficulty。
    Args:
        task (dict): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json

    Return:
        float: 0 ~ 1 之間的浮點數。
    """
    pre_confidence = task['pre-answer']['confidence']
    difficulty_std_r = (1 - pre_confidence) / 3
    difficulty_std_l = pre_confidence / 3
    difficulty_list = list()

    difficulty_2d_r = (pre_confidence + difficulty_std_r * np.random.randn(1, 200))
    difficulty_2d_l = (pre_confidence + difficulty_std_l * np.random.randn(1, 200))
    for difficulty in difficulty_2d_r[0]:
        if difficulty >= pre_confidence and difficulty <= 1:
            difficulty_list.append(difficulty)

        if len(difficulty_list) == 50:
            break

    for difficulty in difficulty_2d_l[0]:
        if difficulty >= 0 and difficulty <= pre_confidence:
            difficulty_list.append(difficulty)

        if len(difficulty_list) == 100:
            break

    difficulty = difficulty_list[random.randint(0, 99)]
    return difficulty


def gen_true_conf(pre_confidence, level_comb, levels_dict, answer_dict):
    """
    利用 pre_confidence 以及 level_comb ，計算出在 level_comb 的組合下，
    最後得到的 confidence 會是多少。

    Args:
        pre_confidence (float): 0 ~ 1 之間的浮點數
        level_comb (list): worker 的組合，像是 ['level1', 'level1', 'level2'] 代表兩個 level 1 的 worker ，以及一個 level 2 的 worker
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json

    Return:
        float: 0 ~ 1 之間的浮點數
    """
    true_conf_direct = pre_confidence
    false_conf_direct = 1 - true_conf_direct
    for level_name in level_comb:
        random_answer_index = random.randint(0, len(answer_dict[level_name])-1)
        answer = answer_dict[level_name][random_answer_index]
        if answer['option'] is True:
            true_conf_direct = true_conf_direct * levels_dict[level_name]['quality']
            false_conf_direct = false_conf_direct * (1 - levels_dict[level_name]['quality'])
        else:
            true_conf_direct = true_conf_direct * (1 - levels_dict[level_name]['quality'])
            false_conf_direct = false_conf_direct * levels_dict[level_name]['quality']
    true_conf = true_conf_direct / (true_conf_direct + false_conf_direct)
    return true_conf


def calculate_expected_revenue(task, levels_dict, level_combs):
    results = list()
    for level_comb in level_combs:
        gte_threshold_cnt = 0
        cost_sum = 0
        for level_name in level_comb:
            cost_sum += levels_dict[level_name]['cost']

        for i in range(MAX_ITERATION):
            true_conf_direct = task['pre-answer']['confidence']
            false_conf_direct = 1 - true_conf_direct

            for level_name in level_comb:
                true_ratio = levels_dict[level_name]['true_ratio']
                random_float = random.uniform(0, 1)
                if random_float <= true_ratio:
                    true_conf_direct = true_conf_direct * levels_dict[level_name]['quality']
                    false_conf_direct = false_conf_direct * (1 - levels_dict[level_name]['quality'])
                else:
                    true_conf_direct = true_conf_direct * (1 - levels_dict[level_name]['quality'])
                    false_conf_direct = false_conf_direct * levels_dict[level_name]['quality']

            true_conf = true_conf_direct / (true_conf_direct + false_conf_direct)
            if true_conf >= THRESHOLD:
                gte_threshold_cnt += 1

        expected_revenue = gte_threshold_cnt / MAX_ITERATION * task['revenue']
        result_dict = {
            "level_comb": level_comb,
            "gte_threshold_cnt": gte_threshold_cnt,
            "expected_revenue": expected_revenue,
            "cost": cost_sum
        }
        results.append(result_dict)
    return results


def micro_optimization(task, levels_dict, max_assign_cnt):
    """
    計算 task 在有作多能指派多少 worker 的數量限制下，每一種 worker 組合能得到的 revenue 期望值各是多少。

    Args:
        task (dict): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        max_assign_cnt (int): 最多能指派多少 worker

    Return:
        list of dictionary: 每一個 dictionary 都包含 level_comb, gte_threshold_cnt, expected_revenue 以及 cost
    """
    difficulty = gen_difficulty(task)
    levels_dict['level1']['true_ratio'] = (
        ALPHA * levels_dict['level1']['quality'] + (1 - ALPHA) * difficulty
    )
    levels_dict['level2']['true_ratio'] = (
        ALPHA * levels_dict['level2']['quality'] + (1 - ALPHA) * difficulty
    )

    level1_max_assign_cnt = len(task['answers']['level1'])
    level2_max_assign_cnt = len(task['answers']['level2'])
    level_combs = list()
    for total_assign_cnt in range(1, max_assign_cnt + 1):
        for level1_assign_cnt in range(total_assign_cnt+1):
            level2_assign_cnt = total_assign_cnt - level1_assign_cnt
            if (level1_assign_cnt > level1_max_assign_cnt or 
                    level2_assign_cnt > level2_max_assign_cnt):
                continue
            level_comb = level1_assign_cnt * ['level1'] + level2_assign_cnt * ['level2']
            level_combs.append(level_comb)

    return calculate_expected_revenue(task, levels_dict, level_combs)

<a name='micro-cp'></a>
### micro cp

In [ ]:
def micro_cp(tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget):
    """
    先將 task 的 revenue 及 pre_confidence 相成，再由高而低的排列，從最大的 task 開始解。

    Args:
        tasks (list): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        tasks_answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json
        max_assign_cnt (int): 最多能指派多少 worker
        budget (int): 預算

    Return:
        tasks (list): 會將每個 task 新增一個 is_solved 的值，如果已經被解掉的 task ， is_solved=True
        output_dict (dict): 包含 revenue_sum, correct_cnt 以及 remaining_budget
    """
    remaining_budget = budget
    revenue_sum = 0
    correct_cnt = 0

    for task in tasks:
        task['is_solved'] = False
        task['origin_exp_revenue'] = task['pre-answer']['confidence'] * task['revenue']

    tasks_order_by_origin_exp_revenue = sorted(
        tasks, key=lambda k: k['origin_exp_revenue'], reverse=True
    )
    for task in tasks_order_by_origin_exp_revenue:
        task_id = task['id']
        pre_confidence = task['pre-answer']['confidence']

        results = micro_optimization(task, levels_dict, max_assign_cnt)
        results_order_by_cost = sorted(results, key=lambda k: k['cost'])
        results_order_by_expected_revenue = sorted(
            results_order_by_cost, key=lambda k: k['expected_revenue'], reverse=True
        )
        for result in results_order_by_expected_revenue:
            if (remaining_budget - result['cost']) < 0:
                continue

            if result['expected_revenue'] == 0:
                continue

            true_conf = gen_true_conf(
                pre_confidence, result['level_comb'],
                levels_dict,
                tasks_answer_dict[task_id]
            )
            task['pre-answer']['confidence'] = true_conf

            if true_conf >= THRESHOLD:
                revenue_sum += task['revenue']
                correct_cnt += 1
                task['is_solved'] = True

            remaining_budget -= result['cost']
            break

    outcome_dict = {
        'revenue_sum': revenue_sum,
        'correct_cnt': correct_cnt,
        'remaining_budget':remaining_budget
    }

    return tasks, outcome_dict

<a name='micro-greedy'></a>
### micro greedy

In [ ]:
def micro_greedy(tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget):
    """
    先將 task 做 revenue 由高而低的排列，從 revenue 最大的 task 開始解。

    Args:
        tasks (list): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        tasks_answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json
        max_assign_cnt (int): 最多能指派多少 worker
        budget (int): 預算

    Return:
        tasks (list): 會將每個 task 新增一個 is_solved 的值，如果已經被解掉的 task ， is_solved=True
        output_dict (dict): 包含 revenue_sum, correct_cnt 以及 remaining_budget
    """
    remaining_budget = budget
    revenue_sum = 0
    correct_cnt = 0

    for task in tasks:
        task['is_solved'] = False

    tasks_order_by_revenue = sorted(tasks, key=lambda k: k['revenue'], reverse=True)
    for task in tasks_order_by_revenue:
        task_id = task['id']
        pre_confidence = task['pre-answer']['confidence']

        results = micro_optimization(task, levels_dict, max_assign_cnt)
        results_order_by_cost = sorted(results, key=lambda k: k['cost'])
        results_order_by_expected_revenue = sorted(
            results_order_by_cost, key=lambda k: k['expected_revenue'], reverse=True
        )
        for result in results_order_by_expected_revenue:
            if (remaining_budget - result['cost']) < 0:
                continue

            if result['expected_revenue'] == 0:
                continue

            true_conf = gen_true_conf(
                pre_confidence,
                result['level_comb'],
                levels_dict,
                tasks_answer_dict[task_id]
            )
            task['pre-answer']['confidence'] = true_conf

            if true_conf >= THRESHOLD:
                revenue_sum += task['revenue']
                correct_cnt += 1
                task['is_solved'] = True

            remaining_budget -= result['cost']
            break

    outcome_dict = {
        'revenue_sum': revenue_sum,
        'correct_cnt': correct_cnt,
        'remaining_budget': remaining_budget
    }
    return tasks, outcome_dict

<a name='micro-mckp'></a>
### micro mckp

In [ ]:
def micro_mckp(tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget):
    """
    將我們的問題套用到多背背問題後的解法，可以參考 http://www2.lssh.tp.edu.tw/~hlf/class-1/lang-c/DP.pdf ，裡面的「P06: 分組的背包問題」。

    Args:
        tasks (list): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        tasks_answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json
        max_assign_cnt (int): 最多能指派多少 worker
        budget (int): 預算

    Return:
        tasks (list): 會將每個 task 新增一個 is_solved 的值，如果已經被解掉的 task ， is_solved=True
        output_dict (dict): 包含 revenue_sum, correct_cnt 以及 remaining_budget
    """
    for task in tasks:
        task['is_solved'] = False

    budget_expected_revenue = [0] * (budget+1)
    budget_expected_correct = [0] * (budget+1)
    budget_do_tasks = list()
    for tmp_budget in range(budget+1):
        budget_do_tasks.append(dict())

    for task in tasks:
        task_id = task['id']
        results = micro_optimization(task, levels_dict, max_assign_cnt)

        for tmp_budget in range(budget, 0, -1):
            for result in results:
                cost = result['cost']
                expected_revenue = result['expected_revenue']

                if tmp_budget - cost < 0:
                    continue

                if expected_revenue == 0:
                    continue

                if (budget_expected_revenue[tmp_budget - cost] + expected_revenue > 
                        budget_expected_revenue[tmp_budget]):

                    budget_expected_revenue[tmp_budget] = (
                        budget_expected_revenue[tmp_budget - cost] + expected_revenue
                    )
                    budget_expected_correct[tmp_budget] = (
                        budget_expected_correct[tmp_budget - cost] + 1
                    )

                    budget_do_tasks[tmp_budget - cost][task_id] = {
                        "level_comb": result['level_comb']
                    }
                    budget_do_tasks[tmp_budget] = copy.deepcopy(budget_do_tasks[tmp_budget - cost])

    revenue_sum = 0
    correct_cnt = 0
    best_budget = budget_expected_revenue.index(max(budget_expected_revenue))
    for task in tasks:
        if budget_do_tasks[best_budget].get(task['id']) is None:
            continue
        task_id = task['id']
        pre_confidence = task['pre-answer']['confidence']
        level_comb = budget_do_tasks[best_budget][task['id']]['level_comb']
        true_conf = gen_true_conf(
            pre_confidence,
            level_comb,
            levels_dict,
            tasks_answer_dict[task_id]
        )
        task['pre-answer']['confidence'] = true_conf
        if true_conf >= THRESHOLD:
            revenue_sum += task['revenue']
            correct_cnt += 1
            task['is_solved'] = True

    outcome_dict = {
        'revenue_sum': revenue_sum,
        'correct_cnt': correct_cnt,
        'remaining_budget': budget - best_budget
    }
    return tasks, outcome_dict

<a name='micro-difficulty'></a>
### micro difficulty

In [ ]:
def majority_voting(answers):
    ans_counter = Counter(answers)
    major_answer = ans_counter.most_common()[0][0]
    return major_answer

def listify_answers(task, level=1):
    if level not in (1, 2):
        raise ValueError('No such level')
    
    level_key = 'level' + str(level)
    return [ans['option'] for ans in task['answers'][level_key]]

def calculate_difficulty(all_answers, major_answer):
    answer_num = len(all_answers)
    different_num = len(all_answers) - all_answers.count(major_answer)
    return different_num / answer_num

In [ ]:
def micro_optimization_difficulty(task, levels_dict, max_assign_cnt, eta=ETA):
    """
    計算 task 在有作多能指派多少 worker 的數量限制下，每一種 worker 組合能得到的 revenue 期望值各是多少。

    Args:
        task (dict): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        max_assign_cnt (int): 最多能指派多少 worker
        eat: Difficulty門檻

    Return:
        list of dictionary: 每一個 dictionary 都包含 level_comb, gte_threshold_cnt, expected_revenue 以及 cost
    """
    difficulty = task['difficulty']
    levels_dict['level1']['true_ratio'] = (
        ALPHA * levels_dict['level1']['quality'] + (1 - ALPHA) * difficulty
    )
    levels_dict['level2']['true_ratio'] = (
        ALPHA * levels_dict['level2']['quality'] + (1 - ALPHA) * difficulty
    )

    level1_max_assign_cnt = len(task['answers']['level1'])
    level2_max_assign_cnt = len(task['answers']['level2'])
    
    choosen_level = 'level1' if difficulty < eta else 'level2'
    max_assign_cnt = min(max_assign_cnt, len(task['answers'][choosen_level]))
    level_combs = [
        [choosen_level] * n
        for n in range(1, max_assign_cnt+1)
    ]
    
    return calculate_expected_revenue(task, levels_dict, level_combs)

In [ ]:
def micro_cp_difficulty(tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget):
    """
    Args:
        tasks (list): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        tasks_answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json
        max_assign_cnt (int): 最多能指派多少 worker
        budget (int): 預算

    Return:
        tasks (list): 會將每個 task 新增一個 is_solved 的值，如果已經被解掉的 task ， is_solved=True
        output_dict (dict): 包含 revenue_sum, correct_cnt 以及 remaining_budget
    """
    remaining_budget = budget
    revenue_sum = 0
    correct_cnt = 0

    for task in tasks:
        task['is_solved'] = False
        task['origin_exp_revenue'] = task['pre-answer']['confidence'] * task['revenue']

        answers = listify_answers(task)        
        y_head = majority_voting(answers)
        task['difficulty'] = calculate_difficulty(answers, y_head)
        
    tasks_order_by_origin_exp_revenue = sorted(
        tasks,
        key=lambda k: k['origin_exp_revenue'],
        reverse=True
    )
    for task in tasks_order_by_origin_exp_revenue:
        task_id = task['id']
        pre_confidence = task['pre-answer']['confidence']

        results = micro_optimization_difficulty(task, levels_dict, max_assign_cnt)
        results_order_by_cost = sorted(results, key=lambda k: k['cost'])
        results_order_by_expected_revenue = sorted(
            results_order_by_cost,
            key=lambda k: k['expected_revenue'],
            reverse=True
        )
        for result in results_order_by_expected_revenue:
            if (remaining_budget - result['cost']) < 0:
                continue

            if result['expected_revenue'] == 0:
                continue

            true_conf = gen_true_conf(
                pre_confidence,
                result['level_comb'],
                levels_dict,
                tasks_answer_dict[task_id]
            )
            task['pre-answer']['confidence'] = true_conf

            if true_conf >= THRESHOLD:
                revenue_sum += task['revenue']
                correct_cnt += 1
                task['is_solved'] = True

            remaining_budget -= result['cost']
            break

    outcome_dict = {
        'revenue_sum': revenue_sum,
        'correct_cnt': correct_cnt,
        'remaining_budget':remaining_budget
    }
    return tasks, outcome_dict

In [ ]:
def micro_greedy_difficulty(tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget):
    """
    先將 task 做 revenue 由高而低的排列，從 revenue 最大的 task 開始解。

    Args:
        tasks (list): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        tasks_answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json
        max_assign_cnt (int): 最多能指派多少 worker
        budget (int): 預算

    Return:
        tasks (list): 會將每個 task 新增一個 is_solved 的值，如果已經被解掉的 task ， is_solved=True
        output_dict (dict): 包含 revenue_sum, correct_cnt 以及 remaining_budget
    """
    remaining_budget = budget
    revenue_sum = int()
    correct_cnt = int()

    for task in tasks:
        task['is_solved'] = False

        answers = listify_answers(task)        
        y_head = majority_voting(answers)
        task['difficulty'] = calculate_difficulty(answers, y_head)

    tasks_order_by_revenue = sorted(tasks, key=lambda k: k['revenue'], reverse=True)
    for task in tasks_order_by_revenue:
        task_id = task['id']
        pre_confidence = task['pre-answer']['confidence']

        results = micro_optimization_difficulty(task, levels_dict, max_assign_cnt)
        results_order_by_cost = sorted(results, key=lambda k: k['cost'])
        results_order_by_expected_revenue = sorted(
            results_order_by_cost, key=lambda k: k['expected_revenue'], reverse=True
        )
        for result in results_order_by_expected_revenue:
            if (remaining_budget - result['cost']) < 0:
                continue

            if result['expected_revenue'] == 0:
                continue

            true_conf = gen_true_conf(
                pre_confidence,
                result['level_comb'],
                levels_dict,
                tasks_answer_dict[task_id]
            )
            task['pre-answer']['confidence'] = true_conf

            if true_conf >= THRESHOLD:
                revenue_sum += task['revenue']
                correct_cnt += 1
                task['is_solved'] = True

            remaining_budget -= result['cost']
            break

    outcome_dict = {
        'revenue_sum': revenue_sum,
        'correct_cnt': correct_cnt,
        'remaining_budget': remaining_budget
    }
    return tasks, outcome_dict

In [ ]:
def micro_mckp_difficulty(tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget):
    """
    將我們的問題套用到多背背問題後的解法，可以參考 http://www2.lssh.tp.edu.tw/~hlf/class-1/lang-c/DP.pdf ，裡面的「P06: 分組的背包問題」。

    Args:
        tasks (list): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        tasks_answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json
        max_assign_cnt (int): 最多能指派多少 worker
        budget (int): 預算

    Return:
        tasks (list): 會將每個 task 新增一個 is_solved 的值，如果已經被解掉的 task ， is_solved=True
        output_dict (dict): 包含 revenue_sum, correct_cnt 以及 remaining_budget
    """
    for task in tasks:
        task['is_solved'] = False
        
        answers = listify_answers(task)        
        y_head = majority_voting(answers)
        task['difficulty'] = calculate_difficulty(answers, y_head)

    budget_expected_revenue = [0] * (budget+1)
    budget_expected_correct = [0] * (budget+1)
    budget_do_tasks = list()
    for tmp_budget in range(budget+1):
        budget_do_tasks.append(dict())

    for task in tasks:
        task_id = task['id']
        results = micro_optimization_difficulty(task, levels_dict, max_assign_cnt)

        for tmp_budget in range(budget, 0, -1):
            for result in results:
                cost = result['cost']
                expected_revenue = result['expected_revenue']

                if tmp_budget - cost < 0:
                    continue

                if expected_revenue == 0:
                    continue

                if (budget_expected_revenue[tmp_budget - cost] + expected_revenue > 
                        budget_expected_revenue[tmp_budget]):

                    budget_expected_revenue[tmp_budget] = (
                        budget_expected_revenue[tmp_budget - cost] + expected_revenue
                    )
                    budget_expected_correct[tmp_budget] = (
                        budget_expected_correct[tmp_budget - cost] + 1
                    )

                    budget_do_tasks[tmp_budget - cost][task_id] = {
                        "level_comb": result['level_comb']
                    }
                    budget_do_tasks[tmp_budget] = copy.deepcopy(budget_do_tasks[tmp_budget - cost])

    revenue_sum = 0
    correct_cnt = 0
    best_budget = budget_expected_revenue.index(max(budget_expected_revenue))
    for task in tasks:
        if budget_do_tasks[best_budget].get(task['id']) is None:
            continue
        task_id = task['id']
        pre_confidence = task['pre-answer']['confidence']
        level_comb = budget_do_tasks[best_budget][task['id']]['level_comb']
        true_conf = gen_true_conf(
            pre_confidence,
            level_comb,
            levels_dict,
            tasks_answer_dict[task_id]
        )
        task['pre-answer']['confidence'] = true_conf
        if true_conf >= THRESHOLD:
            revenue_sum += task['revenue']
            correct_cnt += 1
            task['is_solved'] = True

    outcome_dict = {
        'revenue_sum': revenue_sum,
        'correct_cnt': correct_cnt,
        'remaining_budget': budget - best_budget
    }
    return tasks, outcome_dict

<a name='macro'></a>
## Macro

<a name='macro-utility'></a>
### Utility Functions

In [ ]:
def macro_uniform(tasks, levels_dict, tasks_answer_dict, level_comb):
    """
    將每一個 task 都分配一樣的 worker 組合

    Args:
        tasks (list): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        tasks_answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json
        level_comb (list): worker 的組合，像是 ['level1', 'level1', 'level2'] 代表兩個 level 1 的 worker ，以及一個 level 2 的 worker
    Return:
        tasks (list): 會將每個 task 新增一個 is_solved 的值，如果已經被解掉的 task ， is_solved=True
        output_dict (dict): 包含 revenue_sum, correct_cnt 以及 remaining_budget
    """
    revenue_sum = 0
    correct_cnt = 0

    for task in tasks:
        task['is_solved'] = False
        task['origin_exp_revenue'] = task['pre-answer']['confidence'] * task['revenue']

    tasks_order_by_origin_exp_revenue = sorted(
        tasks, key=lambda k: k['origin_exp_revenue'], reverse=True
    )
    for task in tasks_order_by_origin_exp_revenue:
        task_id = task['id']
        pre_confidence = task['pre-answer']['confidence']

        true_conf = gen_true_conf(
            pre_confidence,
            level_comb,
            levels_dict,
            tasks_answer_dict[task_id]
        )
        task['pre-answer']['confidence'] = true_conf

        if true_conf >= THRESHOLD:
            revenue_sum += task['revenue']
            correct_cnt += 1
            task['is_solved'] = True

    outcome_dict = {
        'revenue_sum': revenue_sum,
        'correct_cnt': correct_cnt,
        'remaining_budget': 0
    }
    return tasks, outcome_dict

#### Task Filters

In [ ]:
def gte_threshold_filter(**kwargs):
    return kwargs.get('new_confidence') >= CONF_THRESHOLD


def keepup_filter(**kwargs):
    return (kwargs.get('new_confidence') - kwargs.get('pre_confidence')) >= 0

<a name='macro-without-multi-process'></a>
### Without multi-process

In [ ]:
def macro(task_filter, micro_alg,
          tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget,
          apply_uniform=False, uniform_initial_comb=None):
    """
    Args:
        task_filter (function): gte_threshold_filter, keepup_filter
        alg (function): micro_greedy, micro_cp or micro_mckp
        tasks (list): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        tasks_answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json
        max_assign_cnt (int): 最多能指派多少 worker
        budget (int): 預算
        apply_uniform (bool): 是否採用uniform strategy
        uniform_initial_comb (list): 採用uniform strategy時的初始worker安排
    Return:
        macro_result (dict): 列出最後全部得到的 revenue, 答對題數, 剩餘資金，以及每一回合得到的 revenue, 答對題數以及剩餘資金
    """
    macro_result = {
        "total_rev": 0.0,
        "total_correct": 0.0,
        "remaining_budget": 0.0,
        "rounds": list()
    }
    for _ in range(MAX_ROUND):
        macro_result['rounds'].append({
            "round_rev": 0.0,
            "round_correct": 0.0,
            "round_remaining": 0.0
        })

    for index in range(SIMULATE_CNT):
        logger.info(f'Round {index+1} Start')
        
        total_rev = 0.0
        total_correct = 0.0
        remaining_budget = budget

        round_tasks = copy.deepcopy(tasks)
        tasks_dict = dict()
        for task in tasks:
            tasks_dict[task['id']] = copy.deepcopy(task)
            tasks_dict[task['id']]['is_solved'] = False

        for round_num in range(MAX_ROUND):
            if apply_uniform and round_num == 0:
                if uniform_initial_comb is None:
                    uniform_initial_comb = ["level1", "level2"]
                    
                cost_sum = 0
                for level_name in uniform_initial_comb:
                    cost_sum += levels_dict[level_name]['cost']
                round_budget = cost_sum * len(round_tasks)
                result_tasks, outcome_dict = macro_uniform(
                    round_tasks, levels_dict, tasks_answer_dict, uniform_initial_comb
                )
            else:
                round_budget = int(remaining_budget/(MAX_ROUND - round_num))

            result_tasks, outcome_dict = micro_alg(
                round_tasks, levels_dict, tasks_answer_dict, max_assign_cnt, round_budget
            )
            next_round_tasks = list()
            for result_task in result_tasks:
                new_confidence = result_task['pre-answer']['confidence']
                pre_confidence = tasks_dict[result_task['id']]['pre-answer']['confidence']
                tasks_dict[result_task['id']]['pre-answer']['confidence'] = new_confidence
                tasks_dict[result_task['id']]['is_solved'] = result_task['is_solved']
                if (new_confidence < THRESHOLD and
                        task_filter(new_confidence=new_confidence,
                                    pre_confidence=pre_confidence)):
                    next_round_tasks.append(result_task)
            round_tasks = next_round_tasks

            total_rev += outcome_dict['revenue_sum']
            total_correct += outcome_dict['correct_cnt']
            remaining_budget = remaining_budget - round_budget + outcome_dict['remaining_budget']

            macro_result['rounds'][round_num]['round_rev'] += outcome_dict['revenue_sum']
            macro_result['rounds'][round_num]['round_correct'] += outcome_dict['correct_cnt']
            macro_result['rounds'][round_num]['round_remaining'] += remaining_budget
        logger.info(f'Round {index+1} End')

        macro_result['total_rev'] += total_rev
        macro_result['total_correct'] += total_correct
        macro_result['remaining_budget'] += remaining_budget

    macro_result['total_rev'] /= SIMULATE_CNT
    macro_result['total_correct'] /= SIMULATE_CNT
    macro_result['remaining_budget'] /= SIMULATE_CNT
    for round_num in range(MAX_ROUND):
        macro_result['rounds'][round_num]['round_rev'] /= SIMULATE_CNT
        macro_result['rounds'][round_num]['round_correct'] /= SIMULATE_CNT
        macro_result['rounds'][round_num]['round_remaining'] /= SIMULATE_CNT
    return macro_result

<a name='macro-with-multi-process'></a>
### With multi-process

In [ ]:
def macro_simulate(task_filter, micro_alg,
                   tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget,
                   apply_uniform=False, uniform_initial_comb=None):
    round_results = list()
    for _ in range(MAX_ROUND):
        round_results.append({
            "round_rev": 0.0,
            "round_correct": 0.0,
            "round_remaining": 0.0
        })

    total_rev = 0.0
    total_correct = 0.0
    remaining_budget = budget

    round_tasks = copy.deepcopy(tasks)
    tasks_dict = dict()
    for task in tasks:
        tasks_dict[task['id']] = copy.deepcopy(task)
        tasks_dict[task['id']]['is_solved'] = False

    for round_num in range(MAX_ROUND):
        if apply_uniform and round_num == 0:
            if uniform_initial_comb is None:
                uniform_initial_comb = ["level1", "level2"]
                    
            cost_sum = 0
            for level_name in uniform_initial_comb:
                cost_sum += levels_dict[level_name]['cost']
            round_budget = cost_sum * len(round_tasks)
            result_tasks, outcome_dict = macro_uniform(
                round_tasks, levels_dict, tasks_answer_dict, uniform_initial_comb
            )
        else:
            round_budget = int(remaining_budget/(MAX_ROUND - round_num))
            
        result_tasks, outcome_dict = micro_alg(
            round_tasks, levels_dict, tasks_answer_dict, max_assign_cnt, round_budget
        )
        next_round_tasks = list()
        for result_task in result_tasks:
            new_confidence = result_task['pre-answer']['confidence']
            pre_confidence = tasks_dict[result_task['id']]['pre-answer']['confidence']
            tasks_dict[result_task['id']]['pre-answer']['confidence'] = new_confidence
            tasks_dict[result_task['id']]['is_solved'] = result_task['is_solved']
            if (new_confidence < THRESHOLD and
                    task_filter(new_confidence=new_confidence,
                                pre_confidence=pre_confidence)):
                next_round_tasks.append(result_task)
        round_tasks = next_round_tasks

        total_rev += outcome_dict['revenue_sum']
        total_correct += outcome_dict['correct_cnt']
        remaining_budget = remaining_budget - round_budget + outcome_dict['remaining_budget']

        round_results[round_num]['round_rev'] += outcome_dict['revenue_sum']
        round_results[round_num]['round_correct'] += outcome_dict['correct_cnt']
        round_results[round_num]['round_remaining'] += remaining_budget

    simulate_result = {
        "total_rev": total_rev,
        "total_correct": total_correct,
        "remaining_budget": remaining_budget,
        "rounds": round_results
    }
    return simulate_result

                
def macro_async(task_filter, micro_alg,
                tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget,
                apply_uniform=False, uniform_initial_comb=None):
    macro_result = {
        "total_rev": 0.0,
        "total_correct": 0.0,
        "remaining_budget": 0.0,
        "rounds": list()
    }
    for round_num in range(MAX_ROUND):
        macro_result['rounds'].append({
            "round_rev": 0.0,
            "round_correct": 0.0,
            "round_remaining": 0.0
        })

    pool = Pool(processes=5)
    simulate_processes = list()

    for index in range(SIMULATE_CNT):
        logger.info(f'Async Round {index+1} Start')
        simulate_processes.append(
            pool.apply_async(
                macro_simulate,
                (task_filter, micro_alg,
                 tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget,
                 apply_uniform, uniform_initial_comb)
            )
        )

    pool.close()
    pool.join()
    
    for simulate_process in simulate_processes:
        simulate_result = simulate_process.get()
        logging.info(simulate_result['total_rev'])
        macro_result['total_rev'] += simulate_result['total_rev']
        macro_result['total_correct'] += simulate_result['total_correct']
        macro_result['remaining_budget'] += simulate_result['remaining_budget']

        for round_num in range(MAX_ROUND):
            macro_result['rounds'][round_num]['round_rev'] += (
                simulate_result['rounds'][round_num]['round_rev']
            )
            macro_result['rounds'][round_num]['round_correct'] += (
                simulate_result['rounds'][round_num]['round_correct']
            )
            macro_result['rounds'][round_num]['round_remaining'] += (
                simulate_result['rounds'][round_num]['round_remaining']
            )

    macro_result['total_rev'] /= SIMULATE_CNT
    macro_result['total_correct'] /= SIMULATE_CNT
    macro_result['remaining_budget'] /= SIMULATE_CNT
    for round_num in range(MAX_ROUND):
        macro_result['rounds'][round_num]['round_rev'] /= SIMULATE_CNT
        macro_result['rounds'][round_num]['round_correct'] /= SIMULATE_CNT
        macro_result['rounds'][round_num]['round_remaining'] /= SIMULATE_CNT
    return macro_result

<a name='random-MAMO'></a>
# Random-MAMO

<a name='micro-random'></a>
## Micro

In [ ]:
def micro_optimization_random(task, levels_dict, max_assign_cnt):
    """
    隨機選擇一種worker組合，計算能得到的revenus期望值。

    Args:
        task (dict): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        max_assign_cnt (int): 最多能指派多少 worker

    Return:
        list of dictionary: 每一個 dictionary 都包含 level_comb, gte_threshold_cnt, expected_revenue 以及 cost
    """
    difficulty = gen_difficulty(task)
    levels_dict['level1']['true_ratio'] = ALPHA * levels_dict['level1']['quality'] + (1 - ALPHA) * difficulty
    levels_dict['level2']['true_ratio'] = ALPHA * levels_dict['level2']['quality'] + (1 - ALPHA) * difficulty

    available_levels = (
        ['level1'] * len(task['answers']['level1']) +
        ['level2'] * len(task['answers']['level2'])
    )
    worker_num = random.randint(1, max_assign_cnt)
    level_combs = set(combinations(available_levels, worker_num))
    level_combs = random.sample(level_combs, 1)
    return calculate_expected_revenue(task, levels_dict, level_combs)

In [ ]:
def micro_random(tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget):
    """
    隨機選擇能解的task

    Args:
        tasks (list): 有 task 資訊的 dictonary ，可參考 Exp-Data/task.json.
        levels_dict (dict): worker 的等級，以及 quality ，可以參考 Exp-Data/levels.json
        tasks_answer_dict (dict): 在真正的 Crowdsourcing 中， level1 及 level2 worker 的回答，可參考 Exp-Data/answers.json
        max_assign_cnt (int): 最多能指派多少 worker
        budget (int): 預算

    Return:
        tasks (list): 會將每個 task 新增一個 is_solved 的值，如果已經被解掉的 task ， is_solved=True
        output_dict (dict): 包含 revenue_sum, correct_cnt 以及 remaining_budget
    """
    remaining_budget = budget
    revenue_sum = 0
    correct_cnt = 0

    for task in tasks:
        task['is_solved'] = False

    random.shuffle(tasks)
    for task in tasks:
        task_id = task['id']
        pre_confidence = task['pre-answer']['confidence']

        results = micro_optimization_random(task, levels_dict, max_assign_cnt)
        for result in results:
            if (remaining_budget - result['cost']) < 0:
                continue

            if result['expected_revenue'] == 0:
                continue

            true_conf = gen_true_conf(
                pre_confidence, result['level_comb'],
                levels_dict,
                tasks_answer_dict[task_id]
            )
            task['pre-answer']['confidence'] = true_conf

            if true_conf >= THRESHOLD:
                revenue_sum += task['revenue']
                correct_cnt += 1
                task['is_solved'] = True

            remaining_budget -= result['cost']
            break

    outcome_dict = {
        'revenue_sum': revenue_sum,
        'correct_cnt': correct_cnt,
        'remaining_budget':remaining_budget
    }

    return tasks, outcome_dict

<a name='macro-random'></a>
## Macro

In [ ]:
def macro_random(tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget):
    macro_result = {
        "total_rev": 0.0,
        "total_correct": 0.0,
        "remaining_budget": 0.0,
        "rounds": list()
    }
    for _ in range(MAX_ROUND):
        macro_result['rounds'].append({
            "round_rev": 0.0,
            "round_correct": 0.0,
            "round_remaining": 0.0
        })

    for index in range(SIMULATE_CNT):
        logger.info(f'Round {index+1} Start')
        
        total_rev = 0.0
        total_correct = 0.0
        remaining_budget = budget

        round_tasks = copy.deepcopy(tasks)
        tasks_dict = dict()
        for task in tasks:
            tasks_dict[task['id']] = copy.deepcopy(task)
            tasks_dict[task['id']]['is_solved'] = False

        for round_num in range(MAX_ROUND):
            round_budget = int(remaining_budget/(MAX_ROUND - round_num))

            result_tasks, outcome_dict = micro_random(
                round_tasks, levels_dict, tasks_answer_dict, max_assign_cnt, round_budget
            )
            next_round_tasks = list()
            for result_task in result_tasks:
                new_confidence = result_task['pre-answer']['confidence']
                pre_confidence = tasks_dict[result_task['id']]['pre-answer']['confidence']
                tasks_dict[result_task['id']]['pre-answer']['confidence'] = new_confidence
                tasks_dict[result_task['id']]['is_solved'] = result_task['is_solved']
                if (new_confidence < THRESHOLD and fake_gen.boolean()):
                    next_round_tasks.append(result_task)
            round_tasks = next_round_tasks

            total_rev += outcome_dict['revenue_sum']
            total_correct += outcome_dict['correct_cnt']
            remaining_budget = remaining_budget - round_budget + outcome_dict['remaining_budget']

            macro_result['rounds'][round_num]['round_rev'] += outcome_dict['revenue_sum']
            macro_result['rounds'][round_num]['round_correct'] += outcome_dict['correct_cnt']
            macro_result['rounds'][round_num]['round_remaining'] += remaining_budget
        logger.info(f'Round {index+1} End')

        macro_result['total_rev'] += total_rev
        macro_result['total_correct'] += total_correct
        macro_result['remaining_budget'] += remaining_budget

    macro_result['total_rev'] /= SIMULATE_CNT
    macro_result['total_correct'] /= SIMULATE_CNT
    macro_result['remaining_budget'] /= SIMULATE_CNT
    for round_num in range(MAX_ROUND):
        macro_result['rounds'][round_num]['round_rev'] /= SIMULATE_CNT
        macro_result['rounds'][round_num]['round_correct'] /= SIMULATE_CNT
        macro_result['rounds'][round_num]['round_remaining'] /= SIMULATE_CNT
    return macro_result

---
<a name='requallo'></a>
# Requallo

In [ ]:
class BayesianRequallo:
    """bayesian requirement"""
    
    def __init__(self, tasks, levels_dict, tasks_answer_dict, budget):
        self.levels_dict = levels_dict
        self.tasks_answer_dict = tasks_answer_dict
        
        self.solved_tasks = list()
        self.unsolved_tasks = deepcopy(tasks)
        self.skipped_tasks = list()
        self.budget = budget
    
    def run(self):
        while self.budget > 0:
            logger.debug(f'-----Iteration start-----')

            max_reward, max_index = -1, None
            for index, task in enumerate(self.unsolved_tasks):
                reward = self._compute_reward(task)
                
                if reward > max_reward:
                    max_reward = reward
                    max_index = index
                    
            if not (max_reward > 0):
                logger.info(f'No reward is greater than 0')
                break

            choosen_task = self.unsolved_tasks[max_index]
            
            # Random choose a worker that is affordable
            affordabale_groups = [
                group_name
                for group_name in task['answers']
                if self.budget >= self.levels_dict[group_name]['cost'] and choosen_task['answers'][group_name]
            ]
                
            if affordabale_groups:
                group_name = random.sample(affordabale_groups, 1)[0]
                new_label = random.sample(choosen_task['answers'][group_name], 1)[0]
                new_label_cost = self.levels_dict[group_name]['cost']
            else:
                logger.warning(f'Label for task {choosen_task} is consumed.')
                
                # Force the requriement to be met since no label can be retrieved
                
                self.skipped_tasks.append(choosen_task)
                self.unsolved_tasks.remove(choosen_task)
                continue
            
            self.budget -= new_label_cost
            if self.budget < 0:
                self.budget += new_label_cost
                logger.info(f'Out of budget.')
                break
            
            # ------
            if 'choosen_labels' not in choosen_task:
                choosen_task['choosen_labels'] = {
                    'level1': [], 
                    'level2': []
                }
                
            choosen_task['choosen_labels'][group_name].append(new_label)
 
            choosen_task['answers'][group_name].remove(new_label)
            # ------
        
            if self._check_task_met_requirement(choosen_task):
                choosen_task['is_solved'] = True
                
                # -------
                self.solved_tasks.append(choosen_task)
                self.unsolved_tasks.remove(choosen_task)
                # -------
                                
            logger.debug(f'-----Iteration end-----')
        logger.info(f'Final Remaining Budget: {self.budget}')
        return self.all_tasks
            
    @property
    def all_tasks(self):
        return self.solved_tasks[:] + self.unsolved_tasks[:]
    
    def _compute_reward(self, task):
        """R(S^t, i^t) = P(y_{i^t} = +1)R_{i^t}^{+1} + P(y_{i^t} = -1)R_{i^t}^{-1} """
        true_label_count, false_label_count = self._calculate_count(task)
        current_value = self._compute_expected_completeness(true_label_count, false_label_count)
        
        level2_label_quality = self.levels_dict['level2']['quality']
        reward_add_true = (
            self._compute_expected_completeness(
                true_label_count*level2_label_quality,
                false_label_count*(1 - level2_label_quality)
            ) +
            current_value
        )
        reward_add_false = (
            self._compute_expected_completeness(
                true_label_count*(1-level2_label_quality),
                false_label_count*level2_label_quality
            ) +
            current_value
        )
        return max(reward_add_true, reward_add_false)
    
    def _calculate_count(self, task):
        pre_confidence = task['pre-answer']['confidence']
        true_count, false_count = pre_confidence, 1 - pre_confidence
        
        if 'choosen_labels' in task:
            for level_name, labels in task['choosen_labels'].items():
                quality = self.levels_dict[level_name]['quality']
                for label in labels:
                    if label['option'] is True:
                        true_count *= quality
                        false_count *= (1 - quality)
                    else:
                        true_count *= (1 - quality)
                        false_count *= quality
        return true_count, false_count
    
    def _compute_expected_completeness(self, true_label_count, false_label_count):
        """V_i(a_i, b_i)"""
        
        total_label_num = true_label_count + false_label_count
        if total_label_num == 0:
            return 0

        return (
            self._compute_difficulty_level(true_label_count, false_label_count) *
            (total_label_num)/self._compute_required_label_num(true_label_count, false_label_count)
            +
            self._compute_difficulty_level(false_label_count, true_label_count) *
            (total_label_num)/self._compute_required_label_num(false_label_count, true_label_count)
        )     
    
    def _compute_difficulty_level(self, expected_label_count, opposite_label_count):
        """P(Z_i = + 1 | a_i, b_i)"""
        
        total_label_num = expected_label_count + opposite_label_count
        
        if expected_label_count == opposite_label_count:
            return 0.5
        else:
            ratio_of_expected_labels = expected_label_count/total_label_num
            
            if expected_label_count > opposite_label_count:
                #  a_i / (a_i + b_i) + b_i / r(b_i)
                return (
                    ratio_of_expected_labels +
                    opposite_label_count/self._compute_required_label_num(expected_label_count, opposite_label_count)
                )
            else:
                # a_i / (a_i + b_i) - a_i / r(a_i)
                return (
                    ratio_of_expected_labels +
                    expected_label_count/self._compute_required_label_num(opposite_label_count, expected_label_count)
                )
            
    def _compute_required_label_num(self, expected_label_count, opposite_label_count):
        """r(a_i, bi|Z_i) >= c
        
        r(a_i) = r(b_i, a_i|Z_i = -1)
        r(b_i) = r(a_i, b_i|Z_i = +1)
        """
        if expected_label_count == 0:
            expected_label_count = 0.25
        
        level2_label_quality = self.levels_dict['level2']['quality']
        num = 1
        const = (
            (opposite_label_count * THRESHOLD) /
            ((1-THRESHOLD) * expected_label_count)
        )
        while level2_label_quality ** num < (1 - level2_label_quality) ** num * const:
            num += 1
        return level2_label_quality**num * expected_label_count

    def _check_task_met_requirement(self, task):
        true_label_count, false_label_count = self._calculate_count(task)
        return self._check_requirement(true_label_count, false_label_count)
        # if true_label_count > false_label_count:
        #     return self._check_requirement(true_label_count, false_label_count)
        # else:
        #     return self._check_requirement(false_label_count, true_label_count)
        
    def _check_requirement(self, expected_label_count, opposite_label_count):
        return expected_label_count/(expected_label_count+opposite_label_count) >= THRESHOLD

In [ ]:
class BayesianRequalloWithRevenueReward:
    """bayesian requirement with revenue reward"""
    
    def __init__(self, tasks, levels_dict, tasks_answer_dict, budget):
        self.levels_dict = levels_dict
        self.tasks_answer_dict = tasks_answer_dict
        
        self.solved_tasks = list()
        self.unsolved_tasks = deepcopy(tasks)
        self.skipped_tasks = list()
        self.budget = budget
    
    def run(self):
        while self.budget > 0:
            logger.debug(f'-----Iteration start-----')

            max_reward, max_index = -1, None
            for index, task in enumerate(self.unsolved_tasks):
                reward = self._compute_reward(task)
                
                if reward > max_reward:
                    max_reward = reward
                    max_index = index
                    
            if not (max_reward > 0):
                logger.debug(f'No reward is greater than 0')
                break

            choosen_task = self.unsolved_tasks[max_index]
            
            # Random choose a worker that is affordable
            affordabale_groups = [
                group_name
                for group_name in task['answers']
                if self.budget >= self.levels_dict[group_name]['cost'] and choosen_task['answers'][group_name]
            ]
                
            if affordabale_groups:
                group_name = random.sample(affordabale_groups, 1)[0]
                new_label = random.sample(choosen_task['answers'][group_name], 1)[0]
                new_label_cost = self.levels_dict[group_name]['cost']
            else:
                logger.debug(f'Label for task {choosen_task} is consumed.')
                
                # Force the requriement to be met since no label can be retrieved
                
                self.skipped_tasks.append(choosen_task)
                self.unsolved_tasks.remove(choosen_task)
                continue
            
            self.budget -= new_label_cost
            if self.budget < 0:
                self.budget += new_label_cost
                logger.debug(f'Out of budget.')
                break
            
            # ------
            if 'choosen_labels' not in choosen_task:
                choosen_task['choosen_labels'] = {
                    'level1': [], 
                    'level2': []
                }
            choosen_task['choosen_labels'][group_name].append(new_label)
            choosen_task['answers'][group_name].remove(new_label)
            # ------
        
            if self._check_task_met_requirement(choosen_task):
                choosen_task['is_solved'] = True
                
                # -------
                self.solved_tasks.append(choosen_task)
                self.unsolved_tasks.remove(choosen_task)
                # -------
                                
            logger.debug(f'-----Iteration end-----')
        logger.debug(f'Final Remaining Budget: {self.budget}')
        return self.all_tasks
            
    @property
    def all_tasks(self):
        return self.solved_tasks[:] + self.unsolved_tasks[:]
    
    def _compute_reward(self, task):
        """R(S^t, i^t) = P(y_{i^t} = +1)R_{i^t}^{+1} + P(y_{i^t} = -1)R_{i^t}^{-1} """
        true_label_count, false_label_count = self._calculate_count(task)
        current_value = self._compute_expected_completeness(true_label_count, false_label_count)
        
        level2_label_quality = self.levels_dict['level2']['quality']
        reward_add_true = (
            self._compute_expected_completeness(
                true_label_count*level2_label_quality,
                false_label_count*(1 - level2_label_quality)
            ) +
            current_value
        )
        return reward_add_true * task['revenue']
    
    def _calculate_count(self, task):
        pre_confidence = task['pre-answer']['confidence']
        true_count, false_count = pre_confidence, 1 - pre_confidence
        
        if 'choosen_labels' in task:
            for level_name, labels in task['choosen_labels'].items():
                quality = self.levels_dict[level_name]['quality']
                for label in labels:
                    if label['option'] is True:
                        true_count *= quality
                        false_count *= (1 - quality)
                    else:
                        true_count *= (1 - quality)
                        false_count *= quality
        return true_count, false_count
    
    def _compute_expected_completeness(self, true_label_count, false_label_count):
        """V_i(a_i, b_i)"""
        
        total_label_num = true_label_count + false_label_count
        if total_label_num == 0:
            return 0

        return (
            self._compute_difficulty_level(true_label_count, false_label_count) *
            (total_label_num)/self._compute_required_label_num(true_label_count, false_label_count)
        )     
    
    def _compute_difficulty_level(self, expected_label_count, opposite_label_count):
        """P(Z_i = + 1 | a_i, b_i)"""
        
        total_label_num = expected_label_count + opposite_label_count
        
        if expected_label_count == opposite_label_count:
            return 0.5
        else:
            ratio_of_expected_labels = expected_label_count/total_label_num
            
            if expected_label_count > opposite_label_count:
                #  a_i / (a_i + b_i) + b_i / r(b_i)
                return (
                    ratio_of_expected_labels +
                    opposite_label_count/self._compute_required_label_num(expected_label_count, opposite_label_count)
                )
            else:
                # a_i / (a_i + b_i) - a_i / r(a_i)
                return (
                    ratio_of_expected_labels +
                    expected_label_count/self._compute_required_label_num(opposite_label_count, expected_label_count)
                )
            
    def _compute_required_label_num(self, expected_label_count, opposite_label_count):
        """r(a_i, bi|Z_i) >= c
        
        r(a_i) = r(b_i, a_i|Z_i = -1)
        r(b_i) = r(a_i, b_i|Z_i = +1)
        """
        if expected_label_count == 0:
            expected_label_count = 0.25
        
        level2_label_quality = self.levels_dict['level2']['quality']
        num = 1
        const = (
            (opposite_label_count * THRESHOLD) /
            ((1-THRESHOLD) * expected_label_count)
        )
        while level2_label_quality ** num < (1 - level2_label_quality) ** num * const:
            num += 1
        return level2_label_quality**num * expected_label_count

    def _check_task_met_requirement(self, task):
        true_label_count, false_label_count = self._calculate_count(task)
        confidence = (true_label_count / (true_label_count+false_label_count))

        return confidence >= THRESHOLD

---

<a name='experiment'></a>
# Experiment

## Load Data

In [ ]:
task_file_path = 'Exp-Data/task.json'
worker_file_path = 'Exp-Data/levels.json'
answer_file_path = 'Exp-Data/answers.json'

In [ ]:
with open(task_file_path, "r") as myfile:
    tasks = ujson.load(myfile)

with open(worker_file_path, "r") as myfile:
    levels_dict = ujson.load(myfile)

with open(answer_file_path, "r") as myfile:
    tasks_answer_dict = ujson.load(myfile)

In [ ]:
sorted_tasks = sorted(tasks, key=lambda x: x['revenue'], reverse=True)

In [ ]:
logging.info('Experiment Start')

In [ ]:
experiment_results = list()

## Random MAMO

In [ ]:
logging.info(f'Random')
result = {
    'method_name': 'random-MAMO'
}

start_time = time.time()
try:
    macro_result = macro_random(tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget)
except Exception as e:
    result['result'] = str(e)
else:
    result['result'] = macro_result
result['time'] = time.time() - start_time 
experiment_results.append(result)

## MAMO

In [ ]:
MICRO_NAME_MAPPING = {
    micro_cp.__name__: 'Greedy-R',
    micro_greedy.__name__: 'Greedy-PR',
    micro_mckp.__name__: 'DP',
    micro_cp_difficulty.__name__: 'Greedy-R-Difficulty',
    micro_greedy_difficulty.__name__: 'Greedy-PR-Difficulty',
    micro_mckp_difficulty.__name__: 'DP-Difficulty'
}

In [ ]:
MACRO_NAME_MAPPING = {
    gte_threshold_filter.__name__: 'ASCC',
    keepup_filter.__name__: 'ASMIC'
}

In [ ]:
available_micro_algorithms = [ 
    micro_cp,
    micro_greedy,   
    micro_mckp,
    micro_cp_difficulty,
    micro_greedy_difficulty,
    micro_mckp_difficulty,
]

In [ ]:
available_macro_filters = [
    gte_threshold_filter,
    keepup_filter,
]

In [ ]:
for macro_filter in available_macro_filters:
    for apply_uniform in [
        True, 
        False
    ]:
        for micro_algorithm in available_micro_algorithms:
            method_name = (
                f'{MICRO_NAME_MAPPING[micro_algorithm.__name__]}'
                f'_{MACRO_NAME_MAPPING[macro_filter.__name__]}'
            )
            if apply_uniform:
                method_name += '_uniform'
                
            logging.info(method_name)
            result = {
                'method_name': method_name
            }
            start_time = time.time()
            try:
                macro_result = macro_async(
                    macro_filter, micro_algorithm,
                    tasks, levels_dict, tasks_answer_dict, max_assign_cnt, budget,
                    apply_uniform
                )
            except Exception as e:
                result['result'] = str(e)
            else:
                result['result'] = macro_result
            result['time'] = time.time() - start_time 
            experiment_results.append(result)

## Requallo

In [ ]:
available_requallo_implementations = [
    BayesianRequalloWithRevenueReward,
]

In [ ]:
for requallo_cls in available_requallo_implementations:
    logger.info(requallo_cls.__name__)
    result = {
        'method_name': 'Bayesian-Requallo',
        "result": {
            "total_rev": 0,
            "total_correct": 0,
            "remaining_budget": 0,
            "rounds":[]
        },
    }
    
    for index in range(5):
        logger.info(f'Round {index+1} Start')
        requallo = requallo_cls(tasks, levels_dict, tasks_answer_dict, budget)
        requallo.run()
        
        result['result']['total_rev'] += sum([task['revenue'] for task in requallo.solved_tasks])
        result['result']['total_correct'] += len(requallo.solved_tasks)
        result['result']['remaining_budget'] += requallo.budget
        logger.info(f'Round {index+1} End')
        
        
    result['result']['total_rev'] /= SIMULATE_CNT
    result['result']['total_correct'] /= SIMULATE_CNT
    result['result']['remaining_budget'] /= SIMULATE_CNT

    experiment_results.append(result)

In [ ]:
for result in experiment_results:
    print(
        result['result']['total_rev'],
        '\t',
        result['method_name'],
    )

In [ ]:
sorted_experiment_result = sorted(experiment_results, key=lambda x: x['result']['total_rev'], reverse=True)

In [ ]:
for result in sorted_experiment_result:
    print(
        result['result']['total_rev'],
        '\t',
        result['method_name'],
    )

In [ ]:
result_filename = f'result-{max_assign_cnt}-{budget}-{int(ETA*10)}.json'
with open(result_filename, 'w') as result_file:
    ujson.dump(experiment_results, result_file, indent=4, ensure_ascii=False)